In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from collections import OrderedDict

page = requests.get('https://www.goodreads.com/quotes/tag/funny')
if page.status_code == 200:
    pageParsed = BeautifulSoup(page.content, 'html5lib')
    
# Define a function that retrieves information about each HTML quote code in a dictionary form.
def extract_data_quote(quote_html):
        quote = quote_html.find('div',{'class':'quoteText'}).get_text().strip().split('\n')[0]
        author = quote_html.find('span',{'class':'authorOrTitle'}).get_text().strip()
        if quote_html.find('div',{'class':'greyText smallText left'}) is not None:
            tags_list = [tag.get_text() for tag in quote_html.find('div',{'class':'greyText smallText left'}).find_all('a')]
            tags = list(OrderedDict.fromkeys(tags_list))
            if 'attributed-no-source' in tags:
                tags.remove('attributed-no-source')
        else:
            tags = None
        data = {'quote':quote, 'author':author, 'tags':tags}
        return data

# Define a function that retrieves all the quotes on a single page. 
def get_quotes_data(page_url):
    page = requests.get(page_url)
    if page.status_code == 200:
        pageParsed = BeautifulSoup(page.content, 'html5lib')
        quotes_html_page = pageParsed.find_all('div',{'class':'quoteDetails'})
        return [extract_data_quote(quote_html) for quote_html in quotes_html_page]


In [3]:
import time

# Initialize timing variables
start_time = time.time()
total_pages = 9549
times = []

# Retrieve data from the first page.
try:
    data = get_quotes_data('https://www.goodreads.com/quotes/tag/funny')
except Exception as e:
    print(f"Failed to retrieve data for page 1. Error: {e}")

# Calculate time taken and add it to times list
end_time = time.time()
elapsed_time = end_time - start_time
times.append(elapsed_time)

# Retrieve data from all other pages.
for i in range(2, total_pages + 1):
    page_start_time = time.time()

    try:
        url = f'https://www.goodreads.com/quotes/tag/funny?page={i}'
        data_current_page = get_quotes_data(url)
    except Exception as e:
        print(f"Failed to retrieve data for page {i}. Error: {e}")
        continue

    # Calculate time taken and add it to times list
    page_end_time = time.time()
    elapsed_time = page_end_time - page_start_time
    times.append(elapsed_time)

    # Estimate remaining time
    avg_time_per_page = sum(times) / len(times)
    remaining_pages = total_pages - i
    estimated_remaining_time = avg_time_per_page * remaining_pages

    # Print progress message
    print(f'Processed page {i} out of {total_pages}. Estimated remaining time: {estimated_remaining_time / 60:.2f} minutes.')


Processed page 2 out of 9549. Estimated remaining time: 235.47 minutes.
Processed page 3 out of 9549. Estimated remaining time: 226.06 minutes.
Processed page 4 out of 9549. Estimated remaining time: 217.85 minutes.
Processed page 5 out of 9549. Estimated remaining time: 213.99 minutes.
Processed page 6 out of 9549. Estimated remaining time: 212.13 minutes.
Processed page 7 out of 9549. Estimated remaining time: 211.52 minutes.
Processed page 8 out of 9549. Estimated remaining time: 209.08 minutes.
Processed page 9 out of 9549. Estimated remaining time: 215.52 minutes.
Processed page 10 out of 9549. Estimated remaining time: 217.94 minutes.
Processed page 11 out of 9549. Estimated remaining time: 215.85 minutes.
Processed page 12 out of 9549. Estimated remaining time: 213.87 minutes.
Processed page 13 out of 9549. Estimated remaining time: 213.99 minutes.
Processed page 14 out of 9549. Estimated remaining time: 213.51 minutes.
Processed page 15 out of 9549. Estimated remaining time: 21

In [4]:

data = data + data_current_page

data_df = pd.DataFrame.from_dict(data)
for i, row in data_df.iterrows():
    if row['tags'] is None:
        data_df = data_df.drop(i)
# Produce the data in a JSON format.
data_df.to_json('C:/Users/khali/OneDrive/GitHub/FineTuningLLM/dataset/funnyQuotes.jsonl',orient="records", lines =True,force_ascii=False)
# Then I used the familiar process to push it to the Hugging Face hub.